# Демонстрация Классификации звонков

## Подключение библиотек, перемненные

In [1]:
import os                              # Функции для работы с файлами
import shutil
import pandas as pd                    # Пандас
import re                              # Работа с регулярными выражениями
import numpy as np
import pickle as pkl
import time
import tensorflow as tf
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.saving import load_model
from tensorflow.keras import utils
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt        # Для прорисовки и визуализации

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
#from sklearn.model_selection import train_test_split
from tqdm import trange, tqdm
from google.colab import data_table
data_table.enable_dataframe_formatter()

!pip install cohere openai
!pip install git+https://github.com/openai/whisper.git
import whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-xenqy3rt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-xenqy3rt
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e4

In [2]:
path_dir = '/content/drive/MyDrive/Media108/'                    # Общий путь для сохранения результатов

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

## Процедуры

In [3]:
def df_to_ohe(df,collist,lst_encoders = {}):
    '''
 Процедура поочередного преобразования колонок из списка через OneHotEncoder в ОНЕ
 С последующей сборкой в единый массив. Параметры:
 df - датафрейм
 collist - список колонок
 Возвращает собранный массив и список энкодеров
    '''
    create_encode = True if len(lst_encoders) == 0 else False
    #print(create_encode)
    list_code = []
    for i in range(len(collist)):
        if create_encode:
            print(f'Формирование OneHotEncoder и кодировка колонки {collist[i]}', end='')
            encoder = OneHotEncoder(handle_unknown='infrequent_if_exist',sparse_output=False)
            encoder.fit(np.array(df[collist[i]].values).reshape(-1, 1))
            lst_encoders[collist[i]] = encoder
            list_code.append(encoder.transform(np.array(df[collist[i]].values).reshape(-1, 1)))
            print(' - Успешно')
        else:
            #print(f'Кодируется колонка {collist[i]}', end='')
            encoder = lst_encoders[collist[i]]
            list_code.append(encoder.transform(np.array(df[collist[i]].values).reshape(-1, 1)))
            #print(' - Успешно')

    x_data = np.hstack(list_code)
    return x_data, lst_encoders

In [19]:
def whisper_sr(df,model='medium',path_audio=''):
    sp_model = whisper.load_model(model)
    options = dict(language='Russian', beam_size=5, best_of=5)
    transcribe_options = dict(task="transcribe", **options)
    for i in trange(len(df),desc='Транскрибация аудиофайлов'):
        filename = df['audiofile'][i]
        if filename == 'Аудиофайл_не_найден':
            rezalt = 'безответа'
        else:
            rezalt = sp_model.transcribe(path_audio+filename,**transcribe_options)['text']

        df.loc[i,['whisper']] = rezalt
#            df.whisper[i] = rezalt
    return df

In [5]:
def file_to_df(df,path_audio='Non'):
    '''
 Функция добаввляет к датафрейму поле "audiofile"
 в которое заносит имя айдиофайла связанного с данной записью
 df - датафрейм
 pats_audio - директория с аудиофайлами
 Возвращает измененный датафрейм.
    '''
    if path_audio == 'Non':
        raise KeyError('Не указан каталог с аудио данными.')

    dict_fls = {}
    for files in os.listdir(path_audio):
        session_in_file = re.findall(r'session_(\d*)_talk',files)
        dict_fls[session_in_file[0]] = files

    df.insert(1,'audiofile',value=np.NaN)
    for k in range(len(df)):
        session = str(df['Идентификатор сессии звонка'][k])
        df.loc[k,['audiofile']] = dict_fls[session]

    df.audiofile = df.audiofile.fillna('Аудиофайл_не_найден')

    return df

In [6]:
def model_pred(df_path, path_audio='',proces=True):
    '''
 Функция для получения предсказания модели
 по переданным данным:
 df_path - Путь к CSV файлу (файл без заголовочной части, если это не возможно - внесем изменения)
 path_audio - Директория а аудиофайлами (если не передана или передана пустой -
              предполагается что транскрибированные данный переданы в датафрейме в колонке "whisper")
 proces - переменная указавающая нужно ли дополнительно обрабатывать колонки
        (очистка теста, заполнение пустот, перевод длительности разговора в сек.)
 Возвращает список с предсказаниями для всего CSV файла и сам датафрейм.
    '''
    # Загрузка предобученной модели.
    model = load_model('/content/drive/MyDrive/Media108/model9_21.h5')
    # Загрузка датафрейма
    df = pd.read_csv(df_path)
    # Загрузка токенайзера и энкодеров необходимых для подготовки данных
    with open('/content/drive/MyDrive/Media108/token_encoders.pkl', 'rb') as f:
        encoders,tokenizer_txt = pkl.load(f)

    # Вызов функции транскрибации аудиозаписей
    if path_audio != '':
        df = file_to_df(df,path_audio=path_audio)
        df.insert(2,'whisper',value='безответа')    # Добавление колонки для транскрибированного текста
        df = whisper_sr(df,model='medium',path_audio=path_audio)   # Типы моделей по усложнению: 'tiny','base','small','medium','large' под вопросом: 'large-v2','large-v3'

    # Очистка текстовой колонки
    df['whisper'] = df['whisper'].fillna('безответа')    # Заполнение пустот
    df['whisper'] = df['whisper'].apply(lambda s: re.sub("[^А-Яа-я0-9 ]", "", s.lower()))
    # Очистка тех строк где после предыдущей операции остались одни пробелы
    df['whisper'] = df['whisper'].apply(lambda s: s.strip())
    # Заменим опустевшие ячейки текстом "безответа"
    df['whisper'] = np.where(df['whisper'].str.len() < 5,'безответа',df['whisper'])
    # Заменим буквы 'ё' на 'е'
    df['whisper'] = df['whisper'].str.replace('ё', 'е', regex=False)

    # Получение матрицы BOW для колонки с расшифровками
    txt_list = df['whisper'].tolist()  # Список расшифровок звонков
    txt_bow = tokenizer_txt.texts_to_matrix(txt_list)

    # Очистка колонок для ОНЕ
    if proces:
        tp_of_ab = df['Тип посетителя'].to_list()
        for i in range(len(tp_of_ab)):
            tp_of_ab[i] = tp_of_ab[i].replace('{','').replace('}','')
        df['Тип посетителя'] = tp_of_ab

    if proces:
        prk = df['Первая рекламная кампания'].to_list()
        for i in range(len(prk)):
            prk[i] = prk[i].replace('{','').replace('}','')
        df['Первая рекламная кампания'] = prk

    # Получение матрицы ОНЕ
    col_to_ohe = ['Статус','Тип','Сайт','Тип посетителя','Сценарий','Операции','Тип устройства','Первая рекламная кампания']
    df[col_to_ohe] = df[col_to_ohe].fillna('Нет данных')   # Дозаполним незаполненные данные
    OHE_data, encoders = df_to_ohe(df,col_to_ohe,lst_encoders = encoders)
    OHE_data = OHE_data.astype('float32')

    # Подготовка Числовых колонок
    if proces:
        df['Чистая длительность разговора'] = df['Чистая длительность разговора'].fillna('00-00-00')
        length_talk = df['Чистая длительность разговора'].to_list()
        len_sec = list()
        for lnt in length_talk:
            match_tlk = re.findall(r'(\d\d)', lnt)
            len_sec.append(int(match_tlk[0])*3600+int(match_tlk[1])*60+int(match_tlk[2]))
        df['Чистая длительность разговора'] = len_sec

        length_talk = df['Длительность звонка'].to_list()
        len_sec = list()
        for lnt in length_talk:
            match_tlk = re.findall(r'(\d\d)', lnt)
            len_sec.append(int(match_tlk[0])*3600+int(match_tlk[1])*60+int(match_tlk[2]))

        df['Длительность звонка'] = len_sec

    # Массив из числовых колонок
    digit_col = ['Чистая длительность разговора','Номер обращения','ID посетителя']
    #digit_col = ['Чистая длительность разговора','Длительность звонка']
    df[digit_col] = df[digit_col].fillna(0)       # Дозаполнение если есть незаполненные
    df['ID посетителя'] = df['ID посетителя']/100000000       # Понижение разрядноси колонки для более плавной нормализации
    x_data = np.array(df[digit_col].values)
    x_data = x_data.astype('float32')
    # Нормализация массива
    max_val = x_data.max()
    x_data = x_data / max_val

    y_pred = model.predict({'input_x1':txt_bow,'input_x2':OHE_data,'input_x3':x_data}, batch_size=txt_bow.shape[0])

    df.insert(3,'y_pred',value = np.rint(y_pred))

    df['predict'] = np.where(df.y_pred==1.0,'Целевой','Не целевой')

    return df

## демонстрация работы моддели

### Подготовка датасета с незаполненными тегами и аудио файлов

In [20]:
df_1 = pd.read_csv(path_dir+'df_no_teg.csv')
df_2 = df_1[:50].copy()
df_2.drop(columns=['Имя файла','whisper'],axis=1,inplace=True)
df_2.to_csv(path_dir+'df_to_test.csv',index=False)


### Проверка работы модели на подготовленных данных

In [21]:
s_time = time.time()
df = model_pred(path_dir+'df_to_test.csv',path_audio=path_dir+'AUDIO/',proces=True)
#y_pred = np.array(df.y_pred.values,dtype='float32')
#target = np.array(df.target.values,dtype='float32')
time_all = int(time.time()-s_time)
print(f'Выполнение завершено. Длительность: {int(time_all//60)} мин и {int(time_all%60)} сек.')

Транскрибация аудиофайлов: 100%|██████████| 50/50 [39:17<00:00, 47.15s/it]


1/1 [==============================] - 0s 323ms/step
Выполнение завершено. Длительность: 39 мин и 46 сек.


In [10]:
df.predict.value_counts()

Целевой       30
Не целевой    20
Name: predict, dtype: int64

In [11]:
df[['y_pred','predict','whisper']]

,y_pred,predict,whisper
0,0.0,Не целевой,здравствуйте вы позвонили в компанию пожалуй...
1,1.0,Целевой,здравствуйте вы позвонили в компанию пожалуй...
2,0.0,Не целевой,здравствуйте вы позвонили в компанию пожалуй...
3,0.0,Не целевой,здравствуйте вы позвонили в компанию пожалуй...
4,1.0,Целевой,здравствуйте вы позвонили компании пожалуйст...
5,1.0,Целевой,здравствуйте вы позвонили компанию пожалуйст...
6,1.0,Целевой,здравствуйте вы позвонили в компанию пожалуй...
7,1.0,Целевой,здравствуйте вы позвонили в компанию пожалуй...
8,0.0,Не целевой,здравствуйте вы позвонили в компанию пожалуй...
9,0.0,Не целевой,алло здравствуйте компания мэргрупп меня зовут...


In [12]:
df[['predict','Статус','Тип','Сайт','Тип посетителя','Сценарий','Операции','Тип устройства','Первая рекламная кампания','Чистая длительность разговора','Длительность звонка']][:100]

,predict,Статус,Тип,Сайт,Тип посетителя,Сценарий,Операции,Тип устройства,Первая рекламная кампания,Чистая длительность разговора,Длительность звонка
0,Не целевой,Принятый,Динамический коллтрекинг,pavcity.turbo.site,Вернувшийся,74955141111,Переадресация,Смартфон,Artics | Павелецкая Сити | direct | Яндекс.Дир...,59,60
1,Целевой,Принятый,Аналитика,pavcity.turbo.site,Не заполнен,74955141111,Переадресация,Прочее,Посетители без рекламной кампании,67,69
2,Не целевой,Принятый,Динамический коллтрекинг,pavcity.turbo.site,Вернувшийся,74955141111,Переадресация,Смартфон,Artics | Павелецкая Сити | direct | Яндекс.Дир...,49,50
3,Не целевой,Принятый,Динамический коллтрекинг,pavcity.turbo.site,Вернувшийся,74955141111,Переадресация,Смартфон,Artics | Павелецкая Сити | direct | Яндекс.Дир...,33,34
4,Целевой,Принятый,Аналитика,pavcity.ru,Не заполнен,74955141111,Переадресация,Прочее,Посетители без рекламной кампании,128,130
5,Целевой,Принятый,Аналитика,pavcity.ru,Не заполнен,SIP URI,Переадресация на сотрудника 1,Прочее,Посетители без рекламной кампании,80,81
6,Целевой,Принятый,Аналитика,pavcity.ru,Не заполнен,SIP URI,Переадресация на сотрудника 1,Прочее,Посетители без рекламной кампании,37,38
7,Целевой,Принятый,Аналитика,pavcity.ru,Не заполнен,74955141111,Переадресация,Прочее,Посетители без рекламной кампании,230,231
8,Не целевой,Принятый,Сохранённая переадресация,pavcity.ru,Не заполнен,74955141111,Переадресация,Прочее,Посетители без рекламной кампании,41,42
9,Не целевой,Принятый,Автоперезвон по заявкам,pavcity.ru,Новый,ОЗ (SIP<1),Переадресация на сотрудника 1,ПК,Media108 | Павелецкая Сити | direct | Яндекс.Д...,8,65


In [ ]:
x_time = np.array(df['Чистая длительность разговора'].values)

In [ ]:
print(f'Общая длительность: {int(x_time.sum()//60)} мин и {int(x_time.sum()%60)} сек.')

Общая длительность: 129 мин и 35 сек.


In [ ]:
df.to_csv(path_dir+'df_pred_50.csv',index=False)

# Результаты

Проверка транскрибации 20 файлов длительность почти 45 мин траскрибировалось ~12 мин. на гпу

Транскрибируется файл 2023-07-18_14-54-12.903688_from_79680573141_to_74955141111_session_3098289147_talk.mp3
Транскрибируется файл 2023-07-14_14-11-49.712590_from_79029502470_to_74955141111_session_3095279810_talk.mp3
Транскрибируется файл 2023-06-24_15-26-51.254630_from_79778960807_to_74955141111_session_3045004274_talk.mp3
Транскрибируется файл 2023-06-19_11-27-46.734461_from_79781935179_to_74955141111_session_3032161583_talk.mp3
Транскрибируется файл 2023-07-20_11-37-36.619972_from_79112808417_to_74955141111_session_3100336696_talk.mp3
Транскрибируется файл 2023-07-20_11-37-11.094292_from_79099624838_to_74950216267_session_3105745185_talk.mp3
Транскрибируется файл 2023-07-20_11-36-59.633997_from_79169021186_to_74950216267_session_3100336231_talk.mp3
Транскрибируется файл 2023-07-20_11-33-48.269480_from_79168455509_to_74955141111_session_3099626809_talk.mp3
Транскрибируется файл 2023-07-19_20-51-00.844569_from_79017501146_to_74955141111_session_3098508543_talk.mp3
Транскрибируется файл 2023-07-19_20-18-38.914386_from_74951069644_to_0155649_session_3098758084_talk.mp3

Транскрибируется файл 2023-07-19_20-17-27.757815_from_74951069644_to_0155649_session_3098758009_talk.mp3

Транскрибируется файл 2023-07-19_19-44-09.669864_from_79264271207_to_74955141111_session_3104889660_talk.mp3
Транскрибируется файл 2023-07-19_19-43-25.083244_from_74954871139_to_0155649_session_3098718754_talk.mp3

Транскрибируется файл 2023-07-19_15-29-33.695587_from_79856910926_to_74955141111_session_3098729296_talk.mp3
Транскрибируется файл 2023-07-19_15-15-54.855377_from_79199996432_to_74955141111_session_3098022399_talk.mp3
Транскрибируется файл 2023-07-19_15-03-53.728340_from_79199996432_to_74955141111_session_3097985449_talk.mp3
Транскрибируется файл 2023-07-19_15-03-11.140278_from_79107482069_to_74955141111_session_3097949424_talk.mp3
Транскрибируется файл 2023-07-19_12-17-04.090745_from_79343331086_to_74955141111_session_3103703425_talk.mp3
Транскрибируется файл 2023-07-19_12-09-46.160919_from_74752559035_to_74955141111_session_3097074298_talk.mp3
Транскрибируется файл 2023-07-19_10-20-54.560393_from_79651436000_to_74955141111_session_3099793737_talk.mp3
1/1 [==============================] - 0s 173ms/step

Выполнение завершено. Длительность: 11 мин и 56 сек.